In [ ]:
!pip install yt-dlp tqdm
!sudo apt-get install -y ffmpeg  # nếu môi trường chưa có ffmpeg
# --- CÀI ĐẶT PHỤ THUỘC (chạy 1 lần nếu chưa có) ---
!pip install -q pytube requests tqdm
!apt-get -qq update && apt-get -qq install -y ffmpeg
!pip install -q yt-dlp
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.9 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# --- IMPORT the libary ---
import os, runpy, subprocess, pandas as pd
import pandas as pd
import os
import runpy
import sys
import subprocess

In [ ]:
path = '/content/drive/MyDrive/Ms.Sc/LJMU - Research Phase/Thessis/Dataset/HateClipSeg/segment_level_annotation.csv'
# ==== Change the specific path ====
SCRIPT_PATH = "/content/drive/MyDrive/Ms.Sc/LJMU - Research Phase/Thessis/Dataset/MultiHateClip/video_download.py"
IDS_FILE    = "/content/drive/MyDrive/Ms.Sc/LJMU - Research Phase/Thessis/Dataset/HateClipSeg/data/segment_level/video_ids.txt"
OUT_DIR     = "/content/drive/MyDrive/Ms.Sc/LJMU - Research Phase/Thessis/Dataset/HateClipSeg/data/segment_level"
PLATFORM    = "youtube"   # Media Social Platform

raw = pd.read_csv(path)
raw["Video Id"].to_csv(IDS_FILE, index=False, header=False) #Change the name of column "Video Id" defending on the annotation file
raw.head()

,Video Id,Segment-Level Label,Segment Timestamp
0,bit_0EHvMSiEHVoc,"[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0...","[['0.00', '1.00'], ['1.00', '10.80'], ['10.80'..."
1,bit_0SYLs1h6WtM2,"[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0...","[['0.00', '32.02'], ['32.02', '36.92'], ['36.9..."
2,bit_0ZrexCZJ860o,"[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0...","[['0.00', '6.00'], ['6.00', '11.00'], ['11.00'..."
3,bit_0dcMcI6hYjhw,"[[0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0], [1, 0...","[['0.00', '10.00'], ['10.00', '17.00'], ['17.0..."
4,bit_0e61b9MSI4mx,"[[1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0...","[['0.00', '1.00'], ['1.00', '30.00'], ['30.00'..."


In [ ]:
# --- NAMESPACE ---
ns = runpy.run_path(SCRIPT_PATH)
os.makedirs(OUT_DIR, exist_ok=True)

# --- Video ID list ---
with open(IDS_FILE) as f:
    video_ids = [line.strip() for line in f if line.strip()]
print(f"IDs: {len(video_ids)} | Output: {OUT_DIR}")

# --- WRAPPER FIX BUG + MONKEY-PATCH FUNCTION IN NAMESPACE ---
def fixed_download(video_ids, platform, data_folder):
    os.makedirs(data_folder, exist_ok=True)
    for vid in video_ids:
        out_file = os.path.join(data_folder, f"{vid}.mp4")
        if os.path.exists(out_file):
            continue  # ignore if existing
        try:
            if platform == "youtube":
                # use function download_video(url, file_path)
                url = f"https://www.youtube.com/watch?v={vid}"
                ns["download_video"](url, out_file)
            elif platform == "bilibili":
                ns["request_video"](vid, data_folder)
            else:
                print("Platform không hỗ trợ:", platform)
        except Exception as e:
            print(f"[pytube FAIL] {vid}: {e}")

# Overwrite the function in namespace
ns["download_video_func"] = fixed_download

# --- 1st Download: PYTUBE (fixing function) ---
ns["download_video_func"](video_ids=video_ids, platform=PLATFORM, data_folder=OUT_DIR)
print("Pytube phase done.")

# --- Check & Record the FAIL list ---
downloaded = {os.path.splitext(f)[0] for f in os.listdir(OUT_DIR) if f.lower().endswith(".mp4")}
missing = [vid for vid in video_ids if vid not in downloaded]
print(f"Đã tải: {len(downloaded)} | Thiếu/Fail: {len(missing)}")

fail_list_path = os.path.join(OUT_DIR, "failed_ids.txt")
with open(fail_list_path, "w") as f:
    for vid in missing:
        f.write(vid + "\n")

pd.DataFrame({"video_id": video_ids, "downloaded": [vid in downloaded for vid in video_ids]}).to_csv(
    os.path.join(OUT_DIR, "download_report.csv"), index=False
)
print("Ghi xong failed_ids.txt & download_report.csv")

# --- FALLBACK 2nd Download: yt-dlp ---
def yt_dlp_download(vid, out_dir):
    url = f"https://www.youtube.com/watch?v={vid}"
    cmd = ["yt-dlp", "-f", "mp4", "-N", "4", "-o", os.path.join(out_dir, f"{vid}.%(ext)s"), url]
    return subprocess.call(cmd)

print("Thử fallback bằng yt-dlp cho:", len(missing), "ID")
fails_final = []
for vid in missing:
    # bỏ qua nếu trong lúc chờ đã tải xong
    if os.path.exists(os.path.join(OUT_DIR, f"{vid}.mp4")):
        continue
    rc = yt_dlp_download(vid, OUT_DIR)
    if rc != 0:
        fails_final.append(vid)

print("Fallback xong. Còn lỗi:", len(fails_final))
if fails_final:
    with open(os.path.join(OUT_DIR, "failed_ids_after_ytdlp.txt"), "w") as f:
        for vid in fails_final: f.write(vid+"\n")
    print("Đã lưu failed_ids_after_ytdlp.txt")

IDs: 435 | Output: /content/drive/MyDrive/Ms.Sc/LJMU - Research Phase/Thessis/Dataset/HateClipSeg/data/segment_level
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download complete.
Download 